## Laser Calibration ```gcode``` examples.

Some code to 'calibrate' laser settings to a material.

In [2]:
# Imports and autoreload for development
%load_ext autoreload
%autoreload 2
import gcode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import grbl
cnc = grbl.Grbl(port="/dev/ttyUSB0")

## Point Calibration.

Vary laser power & dwell times in a grid with 5mmx5mm spacing

In [26]:
# Create an empty GCode program.
prog = gcode.GCode(machine=cnc)
prog.G21()
prog.G90()
prog.G92(X=0, Y=0)
prog.G1(F=200)
prog.G0(F=200)
spacing = 5
for y, dwell in enumerate([0.5, 1, 2, 5, 10]):
    for x, power in enumerate([100, 150, 200, 250]):
        prog.G0(X=x*spacing, Y=y*spacing)
        prog.M3(S=power)
        prog.G1(X=x*spacing, Y=y*spacing)
        prog.G4(P=dwell)
        prog.M5()

In [22]:
cnc.status

'<Hold:0|WPos:39.996,95.004,0.000|Bf:15,114|FS:0,0|Ov:100,100,100>'

In [23]:
cnc.reset()

(0, ['', "Grbl 1.1g ['$' for help]", 'ok'])

In [24]:
cnc.status

'<Idle|WPos:39.996,95.004,0.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [25]:
cnc.run(prog)

115.87804841995239

# Line Calibration

Vary feed rate, power and dynamic powering.

In [27]:
gcode.Line()

Line<len=10.0mm, feed=300, power=1>

In [28]:
gcode.hline()

array([[ 0.,  0.],
       [10.,  0.]])

In [29]:
gcode.Line(points=gcode.hline())

Line<len=10.0mm, feed=300, power=1>

In [30]:
gcode.Line(points=gcode.hline(), power=127, feed=100, dynamic_power=True)

Line<len=10.0mm, feed=100, power=127>

In [31]:
gcode.Line(points=gcode.hline(), power=127, feed=100, dynamic_power=False)

Line<len=10.0mm, feed=100, power=127>

In [39]:
gcode.Line(points=gcode.hline(X0=1, Xf=5, Y=2, n_points=2), power=127, feed=100, dynamic_power=False)

Line<len=4.0mm, feed=100, power=127>

In [41]:
gcode.Line(points=gcode.hline(X0=1, Xf=5, Y=2, n_points=9), power=127, feed=100, dynamic_power=False)

Line<len=4.0mm, feed=100, power=127>

In [110]:
def init(**kwargs):
    """
    Basic Machine Init.
    """
    prog = gcode.GCode(**kwargs)
    prog.G21()
    prog.G90()
    prog.G92(X=0, Y=0)
    prog.G1(F=200)
    prog.G0(F=200)
    return prog

In [111]:
# Create an empty GCode program.
prog = init(machine=cnc)

line_spacing=5
# Points To Draw.
line_points = gcode.hline(Xf=10)
for p, power in enumerate([100, 150, 175, 200, 255]):
    prog.G21()
    prog.G90()
    # "New Line"
    prog.G0(X=0, Y=p*line_spacing)
    # Relative
    prog.G91()
    for f, feed in enumerate([100, 200, 300]):        
        for d, dynamic_power in enumerate([True, False]):
            line = gcode.Line(points=line_points, power=power, feed=feed, dynamic_power=dynamic_power)
            prog+=line

In [113]:
cnc.status

'<Idle|WPos:30.004,19.996,0.000|Bf:15,127|FS:0,0>'

In [114]:
cnc.cmd("$C")

['ok', '[MSG:Enabled]', 'ok']

In [115]:
cnc.status

'<Check|WPos:30.004,19.996,0.000|Bf:15,127|FS:0,0>'

In [116]:
def check_mode(self):
    return "Check" in cnc.status

In [117]:
check_mode(cnc)

True

In [124]:
for p, power in enumerate([100, 150, 175, 200, 255]):
    idx=0
    for d, dynamic_power in enumerate([True, False]):
        for f, feed in enumerate([100, 200, 300]):        
            idx+=1
            print(idx-1, d, f, f+d*2)

0 0 0 0
1 0 1 1
2 0 2 2
3 1 0 2
4 1 1 3
5 1 2 4
0 0 0 0
1 0 1 1
2 0 2 2
3 1 0 2
4 1 1 3
5 1 2 4
0 0 0 0
1 0 1 1
2 0 2 2
3 1 0 2
4 1 1 3
5 1 2 4
0 0 0 0
1 0 1 1
2 0 2 2
3 1 0 2
4 1 1 3
5 1 2 4
0 0 0 0
1 0 1 1
2 0 2 2
3 1 0 2
4 1 1 3
5 1 2 4


In [126]:
for p, power in enumerate([100, 150, 175, 200, 255]):
    idx=0
    for d, dynamic_power in enumerate([True, False]):
        for f, feed in enumerate([100, 200, 300]):        
            idx+=1
            print(idx-1, d, f, f+(d+1)*2-1)##!!!

0 0 0 1
1 0 1 2
2 0 2 3
3 1 0 3
4 1 1 4
5 1 2 5
0 0 0 1
1 0 1 2
2 0 2 3
3 1 0 3
4 1 1 4
5 1 2 5
0 0 0 1
1 0 1 2
2 0 2 3
3 1 0 3
4 1 1 4
5 1 2 5
0 0 0 1
1 0 1 2
2 0 2 3
3 1 0 3
4 1 1 4
5 1 2 5
0 0 0 1
1 0 1 2
2 0 2 3
3 1 0 3
4 1 1 4
5 1 2 5


In [137]:
for p, power in enumerate([100, 150, 175, 200, 255]):
    idx=0
    for d, dynamic_power in enumerate([True, False]):
        for f, feed in enumerate([100, 200, 300]):        
            idx+=1
#            idx=f+(d+1)*2-1)##!!!
            hline_cfg = {
                "X0":(idx-1)*spacing,
                "Xf":idx*spacing,
                "Y" :p*line_spacing,
            }
            line_points = gcode.hline(**hline_cfg)
            line = gcode.Line(points=line_points, power=power, feed=feed, dynamic_power=dynamic_power)

In [138]:
line

Line<len=5.0mm, feed=300, power=255>

In [95]:
spacing=5
# Points To Draw.
line_points = gcode.hline(Xf=spacing)

for p, power in enumerate([100, 150, 175, 200, 255]):
    prog.G21()
    prog.G90()
    # "New Line"
    prog.G0(X=0, Y=p*spacing)
    # Relative
    prog.G91()
    for f, feed in enumerate([100, 200, 300]):        
        for d, dynamic_power in enumerate([True, False]):
            line = gcode.Line(points=line_points, power=power, feed=feed, dynamic_power=dynamic_power)
            prog+=line
prog.run()

...